# Title
[]()

In [1]:

import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append(r"/home/silvhua/repositories/notion/src")
from silvhua import *

In [ ]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Copied from previous notebook `2023-12-26 analyze notion data.ipynb`

In [22]:
from wrangling import filter_df_all_conditions

def create_notion_df(
    filename, filepath, split_columns=['Task Name', 'Task Project name']
    ):
    data = load_json(filename, filepath)
    df = pd.DataFrame(data).transpose()
    for split_column in split_columns:
        # Explode the rows based on the values in the split_column
        df = df.explode(split_column)
    
    columns = df.columns.tolist()
    columns.remove('url')
    columns.remove('Name')
    columns.remove('Task Project')
    df.index.name = 'id'
    columns = ['Name'] + columns + ['url', 'id']
    df = df.reset_index()
    df = df[columns]
    df = df.drop_duplicates(subset=['id', 'Task Project name', 'Task Name']) 
    original_length = df.shape
    print(f'Shape before applying filter: {df.shape}')
    filters = {
        'Elapsed': '> 0'
    }
    df['created_time'] = pd.to_datetime(df['created_time']).dt.tz_convert('America/Vancouver')
    filtered_df = filter_df_all_conditions(df, filters, verbose=False, show_indices=False)
    if original_length != filtered_df.shape:
        print(f'\n**Shape after applying filter: {filtered_df.shape}**\n')
    else:
        print('No change in DataFrame shape after applying filter.')
    return filtered_df


filename = 'notion_time_tracking_parsed_quarter_2023-12-27_0045.json'

path = '/home/silvhua/repositories/notion/data/'

df = create_notion_df(filename, path)
df

Shape before applying filter: (852, 15)
	DataFrame shape: (852, 15)
No change in DataFrame shape after applying filter.


,Name,created_time,Task Tag,Flag,Elapsed,Task Name,Task Project tags,Task Parent-task,Task Sub-tasks,Task Project name,start time,end time,Roadmap Item,url,id
0,;2023-11-30 20:26,2023-11-30 20:26:00-08:00,[life],None,3.566667,Solarathon: Retrieval pipeline,[[portfolio]],[None],[None],Solarathon,20:26,24:00,[],https://www.notion.so/2023-11-30-20-26-a7f579a...,a7f579a6-5439-43b9-9e5d-f695889ac88f
1,2023-11-30 16:44;ts,2023-11-30 16:44:00-08:00,[life],None,1.2,Solarathon: Retrieval pipeline,[[portfolio]],[None],[None],Solarathon,16:03,17:15,[],https://www.notion.so/2023-11-30-16-44-ts-3b00...,3b00d2c2-7b81-4443-834b-84ed72b2ec7d
2,2023-11-30 14:53,2023-11-30 14:53:00-08:00,[life],None,1.166667,Solarathon: Retrieval pipeline,[[portfolio]],[None],[None],Solarathon,14:53,16:03,[],https://www.notion.so/2023-11-30-14-53-f93a93d...,f93a93d3-5fbc-4c9d-a5d1-ceee90b098ba
3,2023-11-30 12:25,2023-11-30 12:25:00-08:00,[Ginkgo],None,0.316667,Corresponding with TMA re: shot status,[[Ginkgo work]],[None],[None],Exercise List,12:25,12:44,[],https://www.notion.so/2023-11-30-12-25-6e29150...,6e291503-5d5c-42ea-a86b-7813c4164b05
4,2023-11-30 11:55,2023-11-30 11:55:00-08:00,[life],None,0.6,Solarathon: explore Haystack,[[portfolio]],[None],[None],Solarathon,11:48,12:24,[],https://www.notion.so/2023-11-30-11-55-1b4cc85...,1b4cc854-fde5-4e4b-8e1d-d1c57bb3fc8f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,2023-09-01 13:30,2023-09-01 13:30:00-07:00,[life],None,1.416667,CloudWolf CCP 75% of content,[[tech ed]],[None],[None],AWS Certified Cloud Practitioner certification,13:30,14:55,[],https://www.notion.so/2023-09-01-13-30-33447cd...,33447cd8-f2b7-49f1-b982-3641b7acfa4d
848,2023-09-01 11:59,2023-09-01 11:59:00-07:00,[Ginkgo],None,0.766667,Ginkgo sprint meeting,[[Ginkgo work]],[None],[None],Ginkgo Admin,12:00,12:46,[],https://www.notion.so/2023-09-01-11-59-3e167f5...,3e167f50-52cd-4b0e-afab-36ba82e3efbc
849,;2023-09-01 10:57,2023-09-01 10:57:00-07:00,[Ginkgo],None,0.766667,Look into WriteSonic AI social media tool (Ruo...,[[Ginkgo work]],[None],[None],knowledge library social media,10:57,11:43,[],https://www.notion.so/2023-09-01-10-57-ce0b078...,ce0b0782-e389-470d-9e9d-9743aee2fb37
850,2023-09-01 10:55,2023-09-01 10:55:00-07:00,[life],None,0.333333,create social media content showing my expertise,[[life]],[None],[3d5b9f63-419e-4d4c-904a-f0d66f5951a5],career positioning,10:35,10:55,[],https://www.notion.so/2023-09-01-10-55-68ed3f5...,68ed3f50-a9db-41bb-93d1-5b8dff439750


## Load saved DF from `pipeline_analyze.py` script

In [21]:
df_filename = 'notion_df.sav'
loaded_df = loadpickle(df_filename, path)
loaded_df

	Time completed: 2023-12-30 12:30:06.000081
Dataframe shape:  (852, 15)
DataFrame columns: ['Name', 'created_time', 'Task Tag', 'Flag', 'Elapsed', 'Task Name', 'Task Project tags', 'Task Parent-task', 'Task Sub-tasks', 'Task Project name', 'start time', 'end time', 'Roadmap Item', 'url', 'id']


,Name,created_time,Task Tag,Flag,Elapsed,Task Name,Task Project tags,Task Parent-task,Task Sub-tasks,Task Project name,start time,end time,Roadmap Item,url,id
0,;2023-11-30 20:26,2023-11-30 20:26:00-08:00,[life],None,3.566667,Solarathon: Retrieval pipeline,[[portfolio]],[None],[None],Solarathon,20:26,24:00,[],https://www.notion.so/2023-11-30-20-26-a7f579a...,a7f579a6-5439-43b9-9e5d-f695889ac88f
1,2023-11-30 16:44;ts,2023-11-30 16:44:00-08:00,[life],None,1.2,Solarathon: Retrieval pipeline,[[portfolio]],[None],[None],Solarathon,16:03,17:15,[],https://www.notion.so/2023-11-30-16-44-ts-3b00...,3b00d2c2-7b81-4443-834b-84ed72b2ec7d
2,2023-11-30 14:53,2023-11-30 14:53:00-08:00,[life],None,1.166667,Solarathon: Retrieval pipeline,[[portfolio]],[None],[None],Solarathon,14:53,16:03,[],https://www.notion.so/2023-11-30-14-53-f93a93d...,f93a93d3-5fbc-4c9d-a5d1-ceee90b098ba
3,2023-11-30 12:25,2023-11-30 12:25:00-08:00,[Ginkgo],None,0.316667,Corresponding with TMA re: shot status,[[Ginkgo work]],[None],[None],Exercise List,12:25,12:44,[],https://www.notion.so/2023-11-30-12-25-6e29150...,6e291503-5d5c-42ea-a86b-7813c4164b05
4,2023-11-30 11:55,2023-11-30 11:55:00-08:00,[life],None,0.6,Solarathon: explore Haystack,[[portfolio]],[None],[None],Solarathon,11:48,12:24,[],https://www.notion.so/2023-11-30-11-55-1b4cc85...,1b4cc854-fde5-4e4b-8e1d-d1c57bb3fc8f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,2023-09-01 13:30,2023-09-01 13:30:00-07:00,[life],None,1.416667,CloudWolf CCP 75% of content,[[tech ed]],[None],[None],AWS Certified Cloud Practitioner certification,13:30,14:55,[],https://www.notion.so/2023-09-01-13-30-33447cd...,33447cd8-f2b7-49f1-b982-3641b7acfa4d
848,2023-09-01 11:59,2023-09-01 11:59:00-07:00,[Ginkgo],None,0.766667,Ginkgo sprint meeting,[[Ginkgo work]],[None],[None],Ginkgo Admin,12:00,12:46,[],https://www.notion.so/2023-09-01-11-59-3e167f5...,3e167f50-52cd-4b0e-afab-36ba82e3efbc
849,;2023-09-01 10:57,2023-09-01 10:57:00-07:00,[Ginkgo],None,0.766667,Look into WriteSonic AI social media tool (Ruo...,[[Ginkgo work]],[None],[None],knowledge library social media,10:57,11:43,[],https://www.notion.so/2023-09-01-10-57-ce0b078...,ce0b0782-e389-470d-9e9d-9743aee2fb37
850,2023-09-01 10:55,2023-09-01 10:55:00-07:00,[life],None,0.333333,create social media content showing my expertise,[[life]],[None],[3d5b9f63-419e-4d4c-904a-f0d66f5951a5],career positioning,10:35,10:55,[],https://www.notion.so/2023-09-01-10-55-68ed3f5...,68ed3f50-a9db-41bb-93d1-5b8dff439750


# Create `df_attributes.json`

In [24]:
print(type(loaded_df['created_time'].max()))
print(loaded_df['created_time'].max())
# convert pandas Timestamp to timestamp in ISO 8601 string format
loaded_df['created_time'].max().isoformat()

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2023-11-30 20:26:00-08:00


'2023-11-30T20:26:00-08:00'

In [31]:

def update_df_attributes(
    df, timestamp_column='created_time', df_filename='notion_df.sav', json_filename='df_attributes',
    path='/home/silvhua/repositories/notion/data'
    ):
    json_file = load_json(f'{json_filename}.json', path)
    newest_created_time = df[timestamp_column].max().isoformat()
    oldest_created_time = df[timestamp_column].min().isoformat()
    json_file[df_filename] = {
        'newest_created_time': newest_created_time,
        'oldest_created_time': oldest_created_time
    }
    save_to_json(
        json_file, json_filename, description=None, 
        append_version=False, path=path
        )
    print(f'Oldest created time: {oldest_created_time}')
    print(f'Newest created time: {newest_created_time}')
    print(f'Success! {json_filename} updated with data from {df_filename}.')

update_df_attributes(loaded_df)



Object saved as JSON: /home/silvhua/repositories/notion/data/df_attributes.json
Oldest created time: 2023-09-01T09:48:00-07:00
Newest created time: 2023-11-30T20:26:00-08:00
Success! df_attributes updated with data from notion_df.sav.


# *End of Page*